In [ ]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

submission_file_path = 
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')

X_cols = ['id', 'date']
y_col = 'class4'

y = train_df[y_col]
X = train_df.drop(columns=[y_col] + X_cols)

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', None),
])

grid = [
    {
        "model": [LogisticRegression(max_iter=1000)],
        "model__C": [0.01, 0.1, 1, 10]
    },
    {
        "model": [RandomForestClassifier()],
        "model__n_estimators": [100, 300],
        "model__max_depth": [None, 10, 30]
    }
]

gs = GridSearchCV(pipeline, grid, cv=10, scoring='accuracy', n_jobs=-1)
gs.fit(X, y)

best_model = gs.best_estimator_

X_test = test_df.drop(columns=X_cols)
probas = best_model.predict_proba(X_test)
prediction = best_model.predict(X_test)

result_df = pd.DataFrame({
    "id": test_df["id"],
    "class4": prediction,
    "p": np.max(probas, axis=1)
})

result_df.to_csv('submission.csv', index=False)